In [1]:
import os
import fitz  # pip install --upgrade pip; pip install --upgrade pymupdf
from tqdm import tqdm  # pip install tqdm
from pdfminer.high_level import extract_text
from PyPDF2 import PdfReader
import re
from pikepdf import Pdf, PdfImage
from PIL import Image
import numpy as np
import uuid
import camelot.io as camelot
import json

In [2]:
# table olish
def get_table(path):
    try:
        all_table = []
        tables = camelot.read_pdf(path, pages = 'all')
        for i in range(len(tables)):
            try:
                name = 'images/'+str(uuid.uuid4())+'.csv'
                if tables[i].df.empty == True:
                    table = tables[i].df.to_markdown()
                    all_table.append(table)
            except:
                pass
        text = get_text_to_list(path)
        description = []
        figure = ''
        starter = []
        for i in range(len(text)):
            string = text[i].replace(' ','')
            if string.startswith('Table') and string[5].isdigit():
                figure = text[i]
                description.append(figure)
        if len(all_table)!=0:
            return (all_table, description)
        else:
            return []
    except:
        return []

In [3]:
def get_image_with_description(path):
    try:
        # open the file
        doc = fitz.open(path)
        workdir = 'images'
        # rasm = []
        # print(len(doc))

        output_img_list = []
        for i in range(len(doc)):
            image_list = doc.get_page_images(i, full=True)
            page = doc[i]
            page_no = page.number+1
            if len(image_list) != 0:

                for img in doc.get_page_images(i):
                    xref = img[0]

                    # get bbox, height, width from image info
                    img_info = page.get_image_info(xref)
                    img_bbox = img_info[0]['bbox']

                    img_height = img_info[0]['width']
                    img_width = img_info[0]['height']

                    #img description extraction
                    extracted_text = page.get_text("dict")["blocks"]
                    line_index = 0
                    img_description = ''
                    # img_bool = False
                    np_arr = np.array(extracted_text)
                    for block in extracted_text:
                        line_index += 1
                        if block['bbox'] == img_bbox and len(extracted_text) > line_index:
                            lines = np_arr[line_index]['lines']
                            for line in lines:
                                spans = line['spans']
                                for span in spans:
                                    # if span['text'] == 'Figure':
                                    #     img_bool = True
                                    # if img_bool == True:
                                    #     img_description += span['text']
                                    img_description += span['text']

                    out = f"IMG_{str(uuid.uuid4())}" + '.png'
                    # image = doc.extract_image(xref)
                    pix = fitz.Pixmap(doc, xref)
                    newpix = fitz.Pixmap(fitz.csRGB, pix)
                    newpix.save(os.path.join(workdir, out))
                    #rasm.append(out)

                    # output result in dic
                    if len(img_description)>50 and 'ABSTRACT:' not in img_description:
                        img_dic = {
                                   'image': 'images/'+out,
                                   'image description': img_description}
                        output_img_list.append(img_dic)
                    if len(img_description)>50 and 'ABSTRACT:' in img_description:
                        img_dic = {
                                   'image': 'images/'+out,
                                   'image description': ''}
                        output_img_list.append(img_dic)
                    del img_info[0]
            page_no += 1
        return output_img_list
    except:
        return []

In [4]:
def get_text(path):
    
    text = extract_text(path)
    text = text.replace("\x0c", "\n\n")
    text = text.replace("\xa0", " ")
    text = text.replace("\u2212", " - ")
    text = text.replace("\n\n", " ")
    text = text.replace("\n", " ")
    return text

In [5]:
# textni list ko'rinishida ajratib oladi
def get_text_to_list(path):
    
    text = extract_text(path)
    text = text.replace('■', '')
    text = text.replace("\u2212", "-")
    text = text.replace("\ufb00", "ff")
    text = text.replace('\ufb03', 'ffi')
    text = text.replace('\ufb01', 'fi')
    text = text.replace('\u0301', 'n')
    text = text.replace('\u2265', '≥')
    text = text.replace('\u03bc', 'μ')
    text = text.replace('Received: November 20, 2021\nRevised:\nJanuary 29, 2022\nPublished: March 18, 2022', "")
    text = text.replace("© 2022 American Chemical Society", "")
    text = text.replace("https://doi.org/10.1021/acscatal.1c05341\nACS Catal. 2022, 12, 4078−4091", "")
    text = text.replace("Downloaded via KOREA INST SCIENCE AND TECHNOLOGY on June 10, 2022 at 00:08:48 (UTC).See https://pubs.acs.org/sharingguidelines for options on how to legitimately share published articles.", "")
    text = text.replace("ACS Catalysis\n\npubs.acs.org/acscatalysis\n\nResearch Article", "")
    text = text.replace("\x0c", "\n\n")
    text = text.replace("\xa0", " ")
    text_test = text.split('\n\n')
    # listga olib bo'limlarga ajratamiz
    
    for i in range(len(text_test)):
        text_test[i] = text_test[i].replace("\n", " ")
        if text_test[i].startswith('Dr. J.-H. Kim, M. K. Kim, Dr. S. Bae, Dr. B. Kim, G. S. Park,   Dr. D. H. Won, Dr. D. K. Lee, Dr. W.'):
            text_test[i] = ''
        elif text_test[i].startswith('The ORCID identification number(s) for the author(s) of this article  can be found under https://doi.org/10.1002/smll.202003865.'):
            text_test[i] = ''
        elif text_test[i].startswith('Prof. T. J. Shin UNIST Central Research Facilities Ulsan National Institute of Science and Technology 50, UNIST-gil, Ulsan 44919, Republic of Korea Prof. Y. R. Do Department of Chemistry Kookmin University 77, Jeongneung-ro, Seongbuk-gu, Seoul 02707, Republic of Korea Prof. J. Kim Department of Chemical and Biological Engineering Korea University 145, Anam-ro, Seongbuk-gu, Seoul 02841, Republic of Korea Prof. B. K. Min Graduate School of Energy and Environment Korea University 145, Anam-ro, Seongbuk-gu, Seoul 02841, Republic of Korea'):
            text_test[i] = ''
        text_test[i] = text_test[i].replace('Small 2020, 16, 2003865', '')
        text_test[i] = text_test[i].replace('© 2020 Wiley-VCH GmbH', '')
        text_test[i] = text_test[i].replace('DOI: 10.1002/smll.202003865', '')
        text_test[i] = text_test[i].replace('www.small-journal.com', '')
        text_test[i] = text_test[i].replace('www.advancedsciencenews.com ', '')
        text_test[i] = text_test[i].replace('www.advancedsciencenews.com', '')
        text_test[i] = text_test[i].replace('Received: June 26, 2020 Revised: August 27, 2020 Published online: November 4, 2020', '')
        try:
            text_test[i+1] = text_test[i+1].replace('(n), compared to the other films.', '')
        except:
            pass
        for j in range(1,16):
            text_test[i] = text_test[i].replace('2003865  ('+str(j)+' of 15)', '')
        string = text_test[i].replace(' ','').lower()
        
        if string.startswith('figure') and string[6].isdigit():
            if string[7].isdigit():
                text_test[i] = '@'+string[:8]+'@'
            else:
                text_test[i] = '@'+string[:7]+'@'
        elif string.startswith('fig.') and string[4].isdigit():
            if string[5].isdigit():
                text_test[i] = '@'+string[:6]+'@'
            text_test[i] = '@'+string[:5]+'@'
        elif string.startswith('table') and string[5].isdigit():
            text_test[i+1] = '@'+string[:6]+'@'
        
    return text_test

In [6]:
# title olish
def get_title(path):
    reader = PdfReader(path)
    meta = reader.metadata
    title = meta.title
    if title is None or "Microsoft" in  meta.title or "MICROSOFT" in  meta.title:
        title = ''
        text = get_text_to_list(path)
        for i in range(len(text)):
            if get_abstract(text)[1].startswith(text[i]):
                break
            title+=text[i]+'\n'
            
    return ('title',title)

In [7]:
# author information olish agar author information bo'sh string qaytarsa title bo'imida authors ni ham oladi
def get_author_information(path):
    text = get_text_to_list(path)
    author = ''
    try:
        result = re.search(get_title(path)[1][-20:]+'(.*)'+get_abstract(text)[1][:20], get_text(path))
        author = str(result.group(1))
        author = author.replace('*', '')
        author = author.replace('and', '')
        author = author.replace('∇', '')
    except:
        pass
    author = author.split('Cite This')[0]
    author = author.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    return ('author information', author)


In [8]:
def get_associated_content(path):
    text = get_text(path)
    content = ''
    try:
        content = text.split('ASSOCIATED CONTENT')[1].split('AUTHOR INFORMATION')[0]
        content = content.replace('■', '')
        content = content.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
        content = content.strip()
        
    except:
        pass
    return ('associated content', content)

In [9]:
def get_author(path):
    text = get_text(path)
    author = ''
    try:
        author = text.split('AUTHOR INFORMATION')[1].split('ACKNOWLEDGMENTS')[0]
        author = author.replace('■', '')
        author = author.strip()
    except:
        pass
    author = author.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    return ('author information', author)

In [10]:
# abstract olish
def get_abstract(text):
    abstract =''
    for i in range(len(text)):
        try:
            if text[i].startswith('Abstract') or text[i].startswith('ABSTRACT'):
                abstract = text[i]
                if len(abstract)<12:
                    abstract = text[i+1]
                break
            else:
                try:
                    if 'Introduction' in text[i] or 'INTRODUCTION' in text[i]:
                        abstract = text[i-1]
                        if abstract.startswith('Keyword') or abstract.startswith('KEYWORD'):
                            abstract = text[i-2]
                        break
                except:
                    pass
        except:
            pass
    abstract = abstract.split('KEYWORDS')[0]
    abstract = abstract.replace('ABSTRACT:', '')
    abstract = abstract.strip()
    return ('Abstract',abstract)

In [11]:
# introduction olish
def get_introduction(text):
    intro = ''
    start = 0
    try:
        for i in range(len(text)-2):
            if 'Introduction' in text[i] or 'INTRODUCTION' in text[i]:
                start = 1
                intro = text[i]
            elif start==1:
                intro += text[i]+' '
            if len(text[i+1])>5 and (text[i+1].startswith('2. ') or text[i+1].startswith('2   ') and text[i+1][4].isupper() and text[i+2].startswith('2.1') or text[i+1].startswith('2 ') and text[i+1][2].isupper() or text[i+2].startswith('2.1 ') or text[i+1].startswith('2  ') and text[i+1][3].isupper() and text[i+2].startswith('2.1')):
                intro = intro.replace('  ', ' ')
                intro = intro.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
                break
            
    except:
        pass
    
    return ('Introduction',intro)

In [12]:
# acknowledgements olish
def get_acknowledgements(text):

    acknowledgments = ''
    start = 0
    num = 0
    keys = ['AUTHOR INFORMATION', 'Ethical Considerations', 'References', 'REFERENCES', 'Keywords', 'KEYWORDS', 'Conflict of Interest']
    try:
        for i in range(len(text)):
            if 'ACKNOWLEDGMENT' in text[i] or 'Acknowledgment' in text[i] or'Acknowledgement' in text[i] or 'ACKNOWLEDGEMENT' in text[i] or 'ACKNOWLEDGMENTS' in text[i] or 'Acknowledgments' in text[i] or'Acknowledgements' in text[i] or 'ACKNOWLEDGEMENTS' in text[i]:
                start = 1
            for j in keys:
                if j in text[i] and start==1:
                    num = 1
            if num ==1:
                break
            if start==1:
                acknowledgments += text[i]+' '   
        acknowledgments = acknowledgments.replace('ACKNOWLEDGMENTS','')
        acknowledgments = acknowledgments.replace('Acknowledgments','')
        acknowledgments = acknowledgments.replace('Acknowledgements','')
        acknowledgments = acknowledgments.replace('ACKNOWLEDGEMENTS','')
        acknowledgments = acknowledgments.replace('ACKNOWLEDGMENT','')
        acknowledgments = acknowledgments.replace('Acknowledgment','')
        acknowledgments = acknowledgments.replace('Acknowledgement','')
        acknowledgments = acknowledgments.replace('ACKNOWLEDGEMENT','')
        acknowledgments = acknowledgments.replace('  ',' ')
        acknowledgments = acknowledgments.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
        acknowledgments = acknowledgments.strip()
    except:
        pass
    return ('Acknowledgments',acknowledgments)

In [13]:
# references olish
def get_references(text):
    references = ''
    start = 0
    referen = []
    begin = -1
    keys = [get_keywords(text)[1][-20:], get_acknowledgements(text)[1][-20:], 'AUTHOR INFORMATION', 'ACKNOWLEDGMENTS', 'Acknowledgements', 'Acknowledgments', 'ACKNOWLEDGEMENTS', 'Ethical Considerations', 'Keywords', 'KEYWORDS']
    for i in range(len(text)):
        if 'References' in text[i] or 'REFERENCES' in text[i]:
            start = 1
        if start==1:
            references += text[i]+' '

    try:
        if start != 1:
            for i in range(1,len(text)+1):
                for j in keys:
                    if j in text[-i]:
                        begin = i 
                        break
            for i in range(-begin+1, 0):
                references += text[i]+' '
            if get_keywords(text)[1] in references:
                references = references.split(get_keywords(text)[1][-30:])[1]
            if get_acknowledgements(text)[1] in references:
                references = references.split(get_acknowledgements(text)[1][-30:])[1]
    except:
        pass
    references = references.replace('REFERENCES', '')
    references = references.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    return ('References',references)

In [14]:
# sectionlarni olish
def get_section(path,number):
    start = 0
    text = get_text_to_list(path)
    natija = 1
    section=''
    begin = get_introduction(text)[1].split('I')[0]
    section_name = '' 
    keys = ['AUTHOR INFORMATION', 'Ethical Considerations', 'References', 'REFERENCES', 'Keywords', 'KEYWORDS','ACKNOWLEDGMENTS', 'Acknowledgements', 'Acknowledgments', 'ACKNOWLEDGEMENTS', 'Supporting Information']
    if "1" in begin:
        for i in text:
            if len(section_name)>30:

                for j in range(2,len(section_name.split())):
                    section+=section_name.split()[j]+' '
                section_name = section_name.split()[0]+section_name.split()[1]

            for a in keys:  
                if a in i[:30] and start==1:
                    # print('yes')
                    natija=0
            if natija == 0:
                break
            if start==1 and i.startswith(begin.replace('1', str(number+1))) and i[len(begin)].isupper():
                break
            if start==1:
                section += i + ' '
            elif i.startswith(begin.replace('1', str(number))):
                start = 1
                section_name = i
    for j in range(len(section)):
        if section[j].isupper() or section[j].isdigit():
            section = section[j:]
            break
    section = section.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    return (section_name,section, natija)

In [15]:
def get_body(path):
    textt = get_text(path)
    text = get_text_to_list(path)
    body = textt.split(get_introduction(text)[1][-365:])[1].split('References')[0]
    body = body.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    return ('body',body)

In [16]:
# keywords olish
def get_keywords(text):
    keywords = ''
    start = 0
    keys = ['Supporting Information', 'AUTHOR INFORMATION', 'ACKNOWLEDGMENTS', 'Acknowledgements', 'Acknowledgments', 'ACKNOWLEDGEMENTS', 'Ethical Considerations', 'Introduction', 'INTRODUCTION', 'References', 'REFERENCES']
    for i in range(len(text)):
        if 'Keywords' in text[i] or  'KEYWORDS' in text[i]:
            start = 1
        if start==1:
            keywords += text[i]+' '     
        for j in keys:
            if j in text[i] and start==1:
                break
        if start == 1 and len(text[i])>10 and ('References' not in text[i+1] or 'REFERENCES' not in text[i+1]):
            break
    keywords = keywords.replace('Keywords','')
    keywords = keywords.replace('KEYWORDS:','')
    keywords = keywords.replace('https://doi.org/10.1021/acscatal.1c05341 ACS Catal. 2022, 12, 4078-4091', '')
    
    return ('Keywords',keywords)

In [17]:
def get_json_temp(types,value):
    
    template = set()
    
    block=dict()
    l=list()
    type_value = dict()
    type_value["type"] = types
    type_value["value"] = value
    l.append(type_value)
    block["block"]=l
    
    return block

In [18]:
# json saqlash
def get_json(path):
    text = get_text_to_list(path)
    
    file = dict()
    file["title"] = get_json_temp("text",get_title(path)[1])
    file["author"] = get_json_temp("text",get_author_information(path)[1])   
    file["abstract"] = get_json_temp("text",get_abstract(text)[1])
    try:
        if 'Introduction' in get_introduction(text)[1]:
            introduction_text = get_introduction(text)[1].split('Introduction')[1]
        elif 'INTRODUCTION' in get_introduction(text)[1]:
            introduction_text = get_introduction(text)[1].split('INTRODUCTION')[1]
        file["introduction"] = get_json_temp("text",introduction_text)
        
    except Exception as myEX:
        print(myEX)
    if get_section(path, 2)[1] == '':
        body = get_body(path)[1]
        file["body"] = get_json_temp("text",body)
        
    else:
        for i in range(2, 10):   
            if get_section(path, i)[2] == 0:
                section = get_section(path, i)[1]
                if 'Result' in get_section(path, i)[0] or 'RESULT' in get_section(path, i)[0]:
                    file['result'] =  get_json_temp("text",section)
                elif 'Conclusion' in get_section(path, i)[0] or 'CONCLUSION' in get_section(path, i)[0]:
                    file['conclusion'] =  get_json_temp("text",section)
                elif 'EXPERIMENTAL SECTION' in get_section(path, i)[0] or 'Experimental section' in get_section(path, i)[0]:
                    file['methods'] =  get_json_temp("text",section)
                else:
                    file['method'+str(i)] =  get_json_temp("text",section)
                break
            elif get_section(path, i)[1]!='':
                section = get_section(path, i)[1]
                if 'Result' in get_section(path, i)[0] or 'RESULT' in get_section(path, i)[0]:
                    file['result'] =  get_json_temp("text",section)
                elif 'Conclusion' in get_section(path, i)[0] or 'CONCLUSION' in get_section(path, i)[0]:
                    file['conclusion'] =  get_json_temp("text",section)
                elif 'EXPERIMENTAL SECTION' in get_section(path, i)[0] or 'Experimental Section' in get_section(path, i)[0]:
                    file['methods'] =  get_json_temp("text",section)
                else:
                    file['method'+str(i)] =  get_json_temp("text",section)
                
    file["associated_content"] = get_json_temp("text",get_associated_content(path)[1])               
    file["author information"] = get_json_temp("text",get_author(path)[1].strip())
    file["acknowledgments"] = get_json_temp("text",get_acknowledgements(text)[1])
    file["references"] = get_json_temp("text",get_references(text)[1].strip())
    file["keywords"] = get_json_temp("text",get_keywords(text)[1].strip())
    file['image_description'] = get_json_temp("image",get_image_with_description(path))
    file["tables"] = get_json_temp("table",get_table(path))
    with open('data.json', 'w') as fp:
        json.dump(file, fp)

In [ ]:
get_json('paper.pdf')